Unicode y encodings
=================

Ingeniería Web, 2020

Qué es un archivo de texto?
=======================

**No** existe un estándar para "archivo de texto plano". Si no nunca veríamos caracteres rotos.

Cuando se rompen decimos rápido: "problema de encodings".

Pero si alguien pregunta "qué son los encodings?"... pocos saben explicarlo.

**Mucha** desinformación y mitos. Incluso en tutoriales y videos!!!

Historia de encodings
=================

Las computadoras no saben de letras. Los humanos saben de letras (caracteres).

Todo texto en última instancia se guarda y transmite como bytes.

Hizo falta definir cómo guardar texto en bytes.

Alguien dijo "que la A sea 1000001, que la B sea 1000010, ...".

... pero muchos alguienes eligieron formas diferentes de hacer esto mismo...

... y ni siquiera con el mismo conjunto de caracteres!! (chino vs occidental vs ...)

Encodings
=========

Ej: ASCII

![](./ascii_table_black.png)

Unicode
=======

"Unicode es un encoding"

![](./no.jpg)

Unicode
=======

En un punto, un grupo de gente/empresas decidió estandarizar la lista de caracteres existentes.

Unicode es el resultado de eso: **es un estándar que define una gran lista de caracteres**, con id, nombre, dibujito, reglas de mayúscula/minúscula, y más de data de cada caracter.

Ej: Unicode "Greek Small Letter Pi" https://www.compart.com/en/unicode/U+03C0

Unicode en si mismo **NO** dice cómo representar cada caracter en bytes. (el id NO es eso!)

Unicode vs encodings
==================

Hoy en día, **un encoding es una tabla que dice cómo representar en bytes, cada caracter de Unicode**.

Ej: "El caracter U+03C0 de Unicode (la letra Pi), se representa con 1100111110000000, ..."

Podemos entonces **convertir entre Unicode y bytes, usando los encodings**:

- **Encodeamos** cuando agarramos unicodes y los convertimos a bytes
- **Decodificamos** cuando agarramos bytes y los convertimos a unicodes

![](./encodear_decodear.png)

(imagen de Facundo Batista)

Encodings incompletos
=========

Unicode admite más de un millón de caracteres. Todavía no completo, se van agregando cada tanto.

Los encodings viejos fueron definidos solo para algunos caracteres. Ej: ASCII tiene 128 caracteres, y no se le pueden agregar más.

Algunos encodings más nuevos tampoco incluyen todo unicode.

Solución?
=========

Usar un mejor encoding.

Los UTF-N son los encodings definidos por el grupo Unicode mismo, y cubren **todo** Unicode, entero. UTF-8 es el más usado en occidente (óptimo para nuestros caracteres, usa menos bytes)

Programación y encodings
======================

Nuestros programas quieren lidiar con texto. Y el texto idealmente se manipula como una secuencia de caracteres Unicode, no como bytes.

Pero vamos a necesitar convertir y desconvertir ese texto a bytes.

Todos los lenguajes tienen herramientas para encodear y decodificar texto. Ejemplo con python, pero es muy similar en todos:

In [3]:
texto = "Ñandú cósmico"

bytes_utf8 = texto.encode('utf-8')
bytes_utf16 = texto.encode('utf-16')
bytes_cp1252 = texto.encode('cp1252')

print(len(texto))
print(len(bytes_utf8))
print(len(bytes_utf16))
print(len(bytes_cp1252))

print(bytes_utf8)

13
16
28
13
b'\xc3\x91and\xc3\xba c\xc3\xb3smico'


In [5]:
bytes_ascii = texto.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\xd1' in position 0: ordinal not in range(128)

In [9]:
print("bien decodificado:", bytes_utf8.decode('utf-8'))
print("mal decodificado:", bytes_utf8.decode('cp1252'))
print("mal decodificado:", bytes_utf8.decode('utf-16'))
print("mal decodificado:", bytes_utf8.decode('ascii'))

bien decodificado: Ñandú cósmico
mal decodificado: Ã‘andÃº cÃ³smico
mal decodificado: 釃湡썤₺썣玳業潣


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 0: ordinal not in range(128)

Mejores prácticas
======================

Usar UTF-8 en todo lo que tenemos control nosotros.

Idealmente deberíamos siempre trabajar así:

1. Leemos o nos llegan bytes, los **decodificamos** usando el encoding que se haya usado para generarlos, y nos quedamos con unicodes (texto!).
2. Trabajamos con el texto en unicode (lo modificamos, armamos, etc)
3. **Encodeamos** a bytes al momento de querer guardarlos o enviarlos a algún lado.

In [11]:
import requests

# leo bytes, y los decodifico
pagina_leida = requests.get('http://www.loiprocesos.com')
contenido = pagina_leida.content.decode('utf-8')

# trabajo con unicodes
contenido = contenido.upper().replace('\n', '') + ' es un ñandú'
print(len(contenido), contenido[-25:])

# vuelvo a encodear al guardar data
datos_salida_en_bytes = contenido.encode('cp1252')  # a la db, una request, un archivo, etc

74662 /BODY></HTML> es un ñandú


Vamos a tener que conocer y configurar encodings de:

- Archivos que leamos o escribamos a disco
- Html que entregamos en una response http
- Base de datos
- Las requests y responses http en sí mismas
- Archivos de código (nuestro programa también es texto!)

A veces es más fácil!
=================

- La mayoría de los lenguajes permite algunas conversiones automáticas al leer o escribir archivos
- La mayoría de los ORMs modernos nos dan la data ya decodificada, y encodean al enviar a la DB (ej: Django)

In [12]:
# ej: al abrir archivos en python, definir el encoding y solo nos decodea y encodea

with open('un_archivo.txt', 'r', encoding='utf-8') as archivo_entrada:
    nombre = archivo_entrada.read()  # nos da unicodes! decodea solo
    
nombre = nombre.upper().replace('\n', '') + ' es un ñandú'
print(nombre)

with open('otro_archivo.txt', 'w', encoding='cp1252') as archivo_salida:
    archivo_salida.write(nombre)  # le pasamos unicodes y nos encodea solo!

FISA es un ñandú


Problemas comunes
=================

No sabemos qué encoding tiene un archivo que nos da un tercero.

Solución: pedir que nos digan el encoding, y si no lo saben o no entienden, hay libs para adivinar, no 100% precisas.

"En qué encoding está el archivo que me pasaste?"

"Está en unicode"

"O...k..."

Una response http o su contenido dice que viene en un encoding, y cuando la leemos con ese encoding falla o vemos caracteres raros.

Solución: pedir que lo arreglen, o si no intentar adivinar cuál es el encoding que realmente están usando.

Al leer/escribir de la base de datos o de un archivo, estamos viendo caracteres extraños.

Solución: seguro estamos leyendo o escribiendo con el encoding incorrecto. Usar el encoding correcto. En bases de datos, el problema más comun suele ser que la base de datos está configurada con un encoding, y al hacerle queries interpretamos los resultados con otro encoding.

Al acceder a nuestro sitio web, se ven caracteres extraños.

Solución: probablemente no estamos especificando el encoding de nuestro html, o lo estamos especificando pero estamos generándolo con otro encoding. Usar el encoding correcto y especificarlo en el html.

Bonus: herramienta para adivinar encodings
=================

`pip3 install chardet --user`

(o `pip install chardet` en un virtualenv)

In [13]:
from chardet import detect

print(detect(bytes_utf8))
print(detect(bytes_utf16))
print(detect(bytes_cp1252))

{'encoding': 'utf-8', 'confidence': 0.87625, 'language': ''}
{'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}
{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


No siempre es posible adivinar perfecto :(